In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from public_tests import *

%matplotlib inline

In [ ]:
import pandas as pd


data = {
    "Cap Color": ["Brown", "Brown", "Brown", "Brown", "Brown", "Red", "Red", "Brown", "Red", "Brown"],
    "Stalk Shape": ["Tapering", "Enlarging", "Enlarging", "Enlarging", "Tapering", "Tapering", "Enlarging", "Enlarging", "Tapering", "Enlarging"],
    "Solitary": ["Yes", "Yes", "No", "No", "Yes", "Yes", "No", "Yes", "No", "No"],
    "Edible": [1, 1, 0, 0, 1, 0, 0, 1, 1, 0]
}


df = pd.DataFrame(data)


df


,Cap Color,Stalk Shape,Solitary,Edible
0,Brown,Tapering,Yes,1
1,Brown,Enlarging,Yes,1
2,Brown,Enlarging,No,0
3,Brown,Enlarging,No,0
4,Brown,Tapering,Yes,1
5,Red,Tapering,Yes,0
6,Red,Enlarging,No,0
7,Brown,Enlarging,Yes,1
8,Red,Tapering,No,1
9,Brown,Enlarging,No,0


In [ ]:
for i in df.columns:
    print(i,df[i].unique())

Cap Color ['Brown' 'Red']
Stalk Shape ['Tapering' 'Enlarging']
Solitary ['Yes' 'No']
Edible [1 0]


In [ ]:
df['Cap Color']=df['Cap Color'].replace({'Brown':1,'Red':0})
df['Stalk Shape']=df['Stalk Shape'].replace({'Tapering':1,'Enlarging':0})
df['Solitary']=df['Solitary'].replace({'Yes':1,'No':0})


df

,Cap Color,Stalk Shape,Solitary,Edible
0,1,1,1,1
1,1,0,1,1
2,1,0,0,0
3,1,0,0,0
4,1,1,1,1
5,0,1,1,0
6,0,0,0,0
7,1,0,1,1
8,0,1,0,1
9,1,0,0,0


In [ ]:
y_train=df['Edible']
x_train=df.drop('Edible',axis=1)

print(x_train.shape,y_train.shape)

(10, 3) (10,)


In [ ]:
def compute_entopy(y):
  if(len(y)!=0):
    p0=len(y[y==0])/len(y)
    p1=len(y[y==1])/len(y)
    if(p1==0) | (p0==0):
      return 0
    entropy=-p0*np.log2(p0)-p1*np.log2(p1)
    return entropy
  else:
    return 0

In [ ]:
y=np.array(y_train)

print(compute_entopy(y))

1.0


In [ ]:
x=np.array(x_train)
x.shape

(10, 3)

In [ ]:
def split_dataset(x,node_indices,feature):
  left_indices=[]
  right_indices=[]
  if(len(node_indices)!=0):
    for i in range(len(x)):
      if(i not in node_indices):
        continue
      else:
        if(x[i][feature]==0):
          right_indices.append(i)

        else:
           left_indices.append(i)
    return left_indices,right_indices
  else:
    return [],[]




In [ ]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
feature = 0

left_indices, right_indices = split_dataset(x, root_indices, feature)

print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]


In [ ]:
y[left_indices]

array([1, 1, 0, 0, 1, 1, 0])

In [ ]:
def compute_information_gain(x,y,node_indices,features):
  left_indices,right_indices=split_dataset(x,node_indices,features)
  entropy_left=compute_entopy(y[left_indices])
  entropy_right=compute_entopy(y[right_indices])

  wleft=len(left_indices)/len(node_indices)
  wright=len(right_indices)/len(node_indices)

  entropy_parent=compute_entopy(y)
  information_gain=entropy_parent-((wleft*entropy_left)+(wright*entropy_right))

  return information_gain


In [ ]:
for i in range(3):
  print(compute_information_gain(x,y,root_indices,i))

0.034851554559677034
0.12451124978365313
0.2780719051126377


In [ ]:
x.shape[1]

3

In [ ]:
def best_split_features(x,y,node_indices):
  features=x.shape[1]
  info_gain=np.zeros(features)
  for i in range(features):
    info_gain[i]=compute_information_gain(x,y,node_indices,i)
  return  np.argmax(info_gain)


In [ ]:
print(best_split_features(x,y,root_indices))

2


In [ ]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree.
        current_depth (int):    Current depth. Parameter used during recursive call.

    """

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return

    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = best_split_features(X, y, node_indices)
    tree.append((current_depth, branch_name, best_feature, node_indices))

    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))

    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)

    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [ ]:
build_tree_recursive(x, y, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
